In [1]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib.animation import FuncAnimation

import os
import glob
import pandas as pd
import json
from dotenv import load_dotenv
from functools import partial

In [2]:
def get_db(data_dir):
    json_files = glob.glob(os.path.join(data_dir, "*.json"))
    data_list = []

    # Iterate through the JSON files and read them
    for file in json_files:
        with open(file, 'r') as f:
            data = json.load(f)
            data_list.append(data)

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data_list)
    return df

In [3]:
model = "bruss"
run_id = "abd_test"
load_dotenv()
data_dir = os.getenv("DATA_DIR")
output_dir = os.getenv("OUT_DIR")
output_dir = os.path.join(output_dir, model, run_id)
os.makedirs(output_dir, exist_ok=True)
df0 = get_db(os.path.join(data_dir, model, run_id))
df0['run_id'].unique()

array(['abd_test'], dtype=object)

In [4]:
df = df0.copy()
df = df[df['run_id'] == run_id]

In [8]:
# def filter_blowup(df, threshold=1e3):
#     valid_rows = []
#     for i, row in df.iterrows():
#         ds = nc.Dataset(row["filename"])
#         data = ds.variables["data"][:]
#         if np.isfinite(data).all() and np.max(data) < threshold:
#             valid_rows.append(row)
#     return pd.DataFrame(valid_rows)

# df = filter_blowup(df)
# len(df)

In [13]:
def classify_and_plot(
    df, sigdigits=2, var1="A", var2="B", file="", steady_threshold=1e-3, osc_threshold=1e-2, dev_threshold=1e-2
):
    """
    Classify runs based on behavior and plot relevant metrics (deviation, spatial or time derivative).
    Args:
        df: DataFrame containing run metadata.
        sigdigits: Number of significant digits in plot titles.
        var1, var2: Variables to label axes (e.g., A, B).
        file: Output file to save the figure.
        steady_threshold: Threshold for ||du/dt|| to classify as steady.
        osc_threshold: Threshold for oscillatory behavior.
        dev_threshold: Threshold for deviation from the steady state.

    Returns:
        Updated DataFrame with classification labels.
    """
    plotting = False
    if len(df) == 0:
        return None

    start_frame = 80  # Ignore early frames to avoid transients
    df = df.sort_values(by=[var1, var2])
    df = df.reset_index(drop=True)

    A_count = len(df[var1].unique())
    B_count = int(len(df) / A_count)

    if plotting:
        fig, axes = plt.subplots(A_count, B_count, figsize=(3 * B_count + 1, 5 * A_count))
        axes = np.atleast_2d(axes)

    classifications = []

    for i, row in df.iterrows():
        # print("A=", row["A"], " B=", row["B"])
        ds = nc.Dataset(row["filename"])
        data = ds.variables["data"][:]  # Assume shape [time, spatial, ...]
        steady_state = np.zeros_like(data[0, 0, :, :])
        steady_state[:, 0::2] = row["A"]
        steady_state[:, 1::2] = row["B"] / row["A"]

        deviations = []
        time_derivatives = []

        du_dt = np.gradient(data[0, :, :, 0::2], row["dt"], axis=0)  # Time derivative of u

        for j in range(start_frame, data.shape[1]):
            deviations.append(np.linalg.norm(data[0, j, :, :] - steady_state))
            time_derivatives.append(np.linalg.norm(du_dt[j]))

        final_dev = deviations[-1]
        mean_dev = np.mean(deviations)
        std_dev = np.std(time_derivatives)
        max_derivative = np.max(time_derivatives)

        # print("Mean deviation=", mean_dev, "Final deviation=", final_dev, "Derivative std=", std_dev)

        # New classification scheme
        if final_dev < dev_threshold or (final_dev < 5 * dev_threshold and max_derivative < steady_threshold):
            category = "steady_state"
        elif std_dev > osc_threshold or mean_dev > dev_threshold:
            category = "interesting_behavior"
        else:
            category = "divergent_or_unknown"
        classifications.append(category)

        if plotting:
            values = time_derivatives
            axes[i // B_count, i % B_count].plot(
                np.arange(0, data.shape[1] - start_frame) * row["dt"] * row["Nt"] / row["n_snapshots"],
                values,
            )
            axes[i // B_count, i % B_count].set_title(
                f"{var1}={row[var1]:.{sigdigits}f}\n{var2}={row[var2]:.{sigdigits}f}\n{category}",
                fontsize=6,
            )

    if plotting:
        row = df.iloc[0]
        time = row["dt"] * row["Nt"]
        fig.suptitle(
            f"{row['model'].capitalize()}, Nx={row['Nx']}, dx={row['dx']}, dt={row['dt']}, T={time:.2f}, Time Derivative ||du/dt||",
            fontsize=4 * B_count,
        )
        plt.tight_layout()
        plt.subplots_adjust(top=0.95)

        if file != "":
            fig.savefig(file, dpi=100)
        plt.close()

    df["category"] = classifications
    return df


In [ ]:
Du = 1
Dv = 4
A = 5
B = 12.5
row = df[(df["Du"] == Du) & (df["Dv"] == Dv) & (df["A"] == A) & (df["B"] == B)].iloc[0]
ds = nc.Dataset(row["filename"])
data = ds.variables["data"][:]  # Assume shape [time, spatial, ...]
steady_state = np.zeros_like(data[0, 0, :, :])
steady_state[:, 0::2] = row["A"]
steady_state[:, 1::2] = row["B"] / row["A"]
Nt = data.shape[1]
deviations = []
time_derivatives = []
deviations_mean = []
du_dt = np.gradient(data[0, :, :, 0::2], row["dt"], axis=0)  # Time derivative of u
print(row["dt"])
for j in range(Nt - 50, Nt):
    u = data[0, j, :, :]
    du = steady_state[:, :] - u
    # print("min ", du.mean())
    # print("mean ", du.mean())
    # print("max ", du.max())
    # print("std ", du.std())
    deviations.append(np.linalg.norm(data[0, j, :, :] - steady_state))
    deviations_mean.append((data[0, j, :, :] - steady_state).mean())
    time_derivatives.append(np.linalg.norm(du_dt[j]))

u = data[0, :, :, 0::2]
v = data[0, :, :, 1::2]
time_derivatives

In [18]:
Du = 3
Dv = 54
df_filt = df[(df["Du"] == Du) & (df["Dv"] == Dv)]

df_class = classify_and_plot(df_filt, sigdigits=2, var1="A", var2="B", file="", steady_threshold=1, osc_threshold=1, dev_threshold=1)

for A in sorted(df_class['A'].unique()):
    for B_mult in [1.25, 1.75, 2, 2.5, 3, 4, 5]:
        B=A*B_mult
        cat = df_class[(df_class['A'] == A) & (df_class['B'] == B)].iloc[0]['category']
        s = ""
        if cat == "steady_state":
            s = "SS"
        elif cat == "interesting_behavior":
            s = "I"
        else:
            s = "DU"
        print(f"{s:3}", end=" ")
    print("")

SS  SS  SS  SS  SS  SS  SS  
SS  SS  SS  I   I   I   I   
SS  I   I   I   I   I   I   
I   I   I   I   I   I   I   
I   I   I   I   I   I   I   
I   I   I   I   I   DU  DU  


In [20]:
from collections import defaultdict

Du_values = [1.0, 2.0, 3.0]
D_mult_values = [4, 11, 18]
A_values = [0.1, 0.5, 1, 2, 5, 10]
B_mult_values = [1.25, 1.75, 2, 2.5, 3, 4, 5]

# Initialize dictionaries to store distributions for each parameter
category_distribution = {
    "A": defaultdict(lambda: {"steady_state": 0, "interesting_behavior": 0, "divergent_or_unknown": 0}),
    "B_mult": defaultdict(lambda: {"steady_state": 0, "interesting_behavior": 0, "divergent_or_unknown": 0}),
    "Du": defaultdict(lambda: {"steady_state": 0, "interesting_behavior": 0, "divergent_or_unknown": 0}),
    "D_mult": defaultdict(lambda: {"steady_state": 0, "interesting_behavior": 0, "divergent_or_unknown": 0}),
}

# Iterate over all parameter combinations
for Du in Du_values:
    for D_mult in D_mult_values:
        Dv = Du * D_mult
        df_filt = df[(df["Du"] == Du) & (df["Dv"] == Dv)]

        df_class = classify_and_plot(df_filt, sigdigits=2, var1="A", var2="B", file="", steady_threshold=1, osc_threshold=1, dev_threshold=1)

        # Update category distributions
        for A in sorted(A_values):
            for B_mult in B_mult_values:
                B = A * B_mult
                row = df_class[(df_class["A"] == A) & (df_class["B"] == B)]
                if not row.empty:
                    cat = row.iloc[0]["category"]
                    if cat == "steady_state":
                        category_distribution["A"][A]["steady_state"] += 1
                        category_distribution["B_mult"][B_mult]["steady_state"] += 1
                        category_distribution["Du"][Du]["steady_state"] += 1
                        category_distribution["D_mult"][D_mult]["steady_state"] += 1
                    elif cat == "interesting_behavior":
                        category_distribution["A"][A]["interesting_behavior"] += 1
                        category_distribution["B_mult"][B_mult]["interesting_behavior"] += 1
                        category_distribution["Du"][Du]["interesting_behavior"] += 1
                        category_distribution["D_mult"][D_mult]["interesting_behavior"] += 1
                    else:
                        category_distribution["A"][A]["divergent_or_unknown"] += 1
                        category_distribution["B_mult"][B_mult]["divergent_or_unknown"] += 1
                        category_distribution["Du"][Du]["divergent_or_unknown"] += 1
                        category_distribution["D_mult"][D_mult]["divergent_or_unknown"] += 1

# Print distributions for each parameter
def print_distributions(param_name, distribution):
    print(f"Distributions for {param_name}:")
    for param_value, counts in sorted(distribution.items()):
        total = sum(counts.values())
        print(f"  {param_value}: {counts} (total={total})")
    print("")

print_distributions("A", category_distribution["A"])
print_distributions("B_mult", category_distribution["B_mult"])
print_distributions("Du", category_distribution["Du"])
print_distributions("D_mult", category_distribution["D_mult"])


Distributions for A:
  0.1: {'steady_state': 63, 'interesting_behavior': 0, 'divergent_or_unknown': 0} (total=63)
  0.5: {'steady_state': 31, 'interesting_behavior': 32, 'divergent_or_unknown': 0} (total=63)
  1: {'steady_state': 15, 'interesting_behavior': 48, 'divergent_or_unknown': 0} (total=63)
  2: {'steady_state': 9, 'interesting_behavior': 54, 'divergent_or_unknown': 0} (total=63)
  5: {'steady_state': 12, 'interesting_behavior': 51, 'divergent_or_unknown': 0} (total=63)
  10: {'steady_state': 18, 'interesting_behavior': 41, 'divergent_or_unknown': 4} (total=63)

Distributions for B_mult:
  1.25: {'steady_state': 45, 'interesting_behavior': 9, 'divergent_or_unknown': 0} (total=54)
  1.75: {'steady_state': 30, 'interesting_behavior': 24, 'divergent_or_unknown': 0} (total=54)
  2: {'steady_state': 27, 'interesting_behavior': 27, 'divergent_or_unknown': 0} (total=54)
  2.5: {'steady_state': 16, 'interesting_behavior': 38, 'divergent_or_unknown': 0} (total=54)
  3: {'steady_state': 